# <center>L2 Computational Physics</center>

## <center>Week 3: Differential Equations I</center>

In [ ]:
# usual packages to import
import numpy 
import matplotlib.pyplot as plt
%matplotlib inline

In this notebook, you will generate and plot the decay curve for Iodine-133 analytically and numerically. $^{133}\textrm{I}$ has a half life $t_{1/2}$ of 20.8 hours. This means that half of the nuclei will have decayed after time $t_{1/2}$. Derive the mean lifetime $\tau$ from that information.

In [ ]:
# define a function to calculate the mean lifetime from the half life
def meanLifetime(halfLife):
    result = halfLife/numpy.log(2)
    return result
    raise NotImplementedError()

T_HALF = 20.8
TAU = meanLifetime(T_HALF)


Check your average lifetime:

In [ ]:
assert numpy.isclose(TAU, 30.0080568505)

### The Decay Equation

Implement the function `f` for the differential equation 

$$ \frac{dN}{dt} = f(N,t)$$

to describe the radioactive decay process.

- *Your function should return values using hours as the time unit.*
- *The function should use the constant* `TAU`.

In [ ]:
def f(N, t):
    result = -N/TAU
    return result
    raise NotImplementedError()

Make sure your function works:

In [ ]:
assert numpy.isclose(f(1000, 0), -33.324383681)

In [ ]:
assert numpy.isclose(f(500,4), -16.6621918403833)

Solve this first order, ordinary differential equation analytically. Implement this function below, naming it `analytic`. The function should take an initial number of atoms `N0` at time `t=0`, and a time argument. The function should return nuclei count at the time argument. Make sure the function also works for numpy arrays.

In [ ]:
def analytic(N0, t):
    result = N0*numpy.exp(-t/TAU)
    return result
    raise NotImplementedError()

Check your answer for a single time:

In [ ]:
assert numpy.isclose(analytic(1000, 41.6), 250.0)

In [ ]:
assert numpy.isclose(analytic(1000, numpy.arange(0, 60, 6)), [1000.        ,  818.77471839,  670.39203948,  548.90005334,
                                                              449.4254866 ,  367.97822623,  301.29126855,  246.68967356,
                                                              201.983268  ,  165.37879338]).all()

## Numerically Solving the ODE

We now wish to solve our differential equation numerically. We shall do this using Euler's and RK4 methods.

### Euler's Method

Create a function which takes as its arguments the initial number of atoms, `n0`, the initial time `t0`, the time step, `dt`, and the number of steps to perform, `n_steps`.  This function should return an array of the number of counts at each time step using Euler's method. This array should contain the initial and final values, so the array length should be `n_steps+1` 

In [ ]:
def solve_euler(n0, t0, dt, n_panels):
    x = numpy.array([analytic(n0, t0)])
    for i in range(0,n_panels):
        counts = x[-1]+(-x[-1]/TAU)*dt
        x = numpy.append(x,[counts])
    return x
    raise NotImplementedError()

Try your solution:

In [ ]:
assert len(solve_euler(1000,0, 1, 17)) == 18

In [ ]:
assert numpy.isclose(solve_euler(1000,0, 6, 1), [1000.,  800.05369792]).all()

In [ ]:
assert numpy.isclose(solve_euler(1000, 0, 6, 10), [1000.        ,  800.05369792,  640.08591955,  512.10310692,
                                                409.7099844 ,  327.7899881 ,  262.24959212,  209.81375595,
                                                167.86227132,  134.29883091,  107.4462763 ]).all()

### RK 4 method

Implement the RK4 method in the `solve_RK4` function. The arguments are the same as for `solve_euler`.

In [ ]:
def solve_RK4(n0,t0, dt, nsteps):
    x = numpy.array([analytic(n0, t0)])
    for i in range(0,nsteps):
        K1 = -x[-1]/TAU
        K2 = -(x[-1]+K1*(dt/2))/TAU
        K3 = -(x[-1]+K2*(dt/2))/TAU
        K4 = -(x[-1]+K3*dt)/TAU
        KTOT = (K1+2*K2+2*K3+K4)/6
        ENDPOS = x[-1]+KTOT*dt
        x = numpy.append(x,[ENDPOS])
    return x
    raise NotImplementedError()


In [ ]:
assert len(solve_RK4(1000,0, 1, 17)) == 18

In [ ]:
assert numpy.isclose(solve_RK4(1000,0, 6, 1), [1000.,  818.7773]).all()

In [ ]:
assert numpy.isclose(solve_RK4(1000, 0, 6, 10), [
    1000.,
    818.77729521,  
    670.39625915,  
    548.90523578,
    449.43114428,  
    367.9840167,  
    301.29695787,  
    246.69510822, 
    201.98835345,  
    165.3834777,  
    135.41223655]).all()

## Plotting task

Create a plot to show that the RK4 method has an error that scales better with the number of steps than the Euler method. (click on the "+" button to create new cells.) 


In [ ]:
Time = 10 # Hours to let the decay go on
N0 = 1000 # Initial number of unstable particles
#As long as it is positive, the value of these variables is irrelevant to my intent

In [ ]:
def EULERROR(npanels):
    dt = Time/npanels
    x = abs(analytic(N0, Time) - solve_euler(N0, 0, dt, npanels)[-1])/(analytic(N0, Time))
    return x
    raise NotImplementedError()

In [ ]:
def RK4ERROR(npanels):
    dt = Time/npanels
    x = abs(analytic(N0, Time) - solve_RK4(N0, 0, dt, npanels)[-1])/(analytic(N0, Time))
    return x
    raise NotImplementedError()

In [ ]:
Xcoords1 = numpy.array([2,4,8,16,32,64,128,256,512])
Ycoords1 = []
Ycoords2 = []
for i in Xcoords1:
    Ycoords1.append(EULERROR(i))
for i in Xcoords1:
    Ycoords2.append(RK4ERROR(i))
plt.loglog(Xcoords1,Ycoords1, '-',label="Euler's method")
plt.loglog(Xcoords1,Ycoords2, '-',label="Runge-Kutta method")
plt.legend(loc='best')
plt.title("Fractional error of decay function approximated with RK4 and Euler's method against the number of panels used")
plt.xlabel("Number of panels used")
plt.ylabel("Fractional Error")